#### Configuring setup

In [ ]:
pip install lightgbm xgboost reverse_geocoder haversine

In [42]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import reverse_geocoder as revgc
from haversine import haversine
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

pd.set_option('display.max_columns', None, 'display.max_rows', None)

In [43]:
## Reading the data
train = pd.read_csv('Data/W23P2_train.csv')
test = pd.read_csv('Data/W23P2_test.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

In [44]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
0,medium,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,medium,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,medium,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,medium,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,high,1.0,1,40.7645,-73.9840,2500,260 W 54 St.,0,0,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
amenities = ['Dining.Room', 'Laundry.in.Building', 'Dishwasher', 'Elevator', 'Laundry.in.Unit', 'Swimming.Pool', 'Roof.Deck', 'Fitness.Center', 
             'High.Speed.Internet', 'Parking.Space', 'Laundry.In.Building', 'Garage', 'Laundry.In.Unit', 'Fireplace', 'Storage', 'dishwasher',
             'LAUNDRY', 'Balcony', 'Furnished', 'Terrace', 'Stainless.Steel.Appliances', 'Central.A.C', 'Gym.Fitness', 'Pool', 
             'Washer.in.Unit', 'Dryer.in.Unit', 'Laundry.Room', 'elevator', 'pool', 'Patio', 'Roof.deck', 'Laundry', 'storage', 
            'Air.conditioning', 'On.site.laundry', 'WiFi', 'Parking', 'EAT.IN.KITCHEN', 'Sauna', 'Eat.In.Kitchen', 'On.site.Garage', 'Bike.room', 
            'balcony', 'Walk.in.Closet.s.', 'On.site.Laundry', 'assigned.parking.space', 'Washer.Dryer', 'Indoor.Pool',
       'Residents.Lounge', 'WiFi.Access', 'Common.parking.Garage', 'Courtyard', 'Lounge.room',
       'Childrens.Playroom', 'Basement.Storage', 'Full.Service.Garage',
       'Health.Club', 'Washer.Dryer.in.building', 'Children.s.Playroom',
       'Outdoor.Entertainment.Space', 'In.Unit.Washer.Dryer', 'Bike.Room', 'On.site.Parking.Lot', 'Cable.Satellite.TV',
       'Microwave', 'Decorative.Fireplace', 'terrace', 'Gym', 'Washer...Dryer', 'patio', 'Washer.Dryer.in.Unit', 'gym', 'Gym.In.Building', 'Common.garden']

pets = ['Dogs.Allowed', 'Cats.Allowed', 'No.pets', 'Pet.Friendly', 'Pets.on.approval']

safety = ['Doorman', 'Concierge', 'Concierge.Service', 'Full.time.doorman', 'LIVE.IN.SUPER', 'Live.in.superintendent', 
         'X24.7.Concierge', 'Virtual.Doorman', 'FT.Doorman', 'Live.In.Superintendent',
       'Live.in.Super', 'doorman', 'Live.In.Super']

privacy = ['Private.Outdoor.Space', 'private.outdoor.space', 'Private.parking', 'Sublet', 'private.balcony', 'Private.balcony', 
          'Private.Deck', 'Private.outdoor.space']

niceness = ['Hardwood.Floors', 'New.Construction', 'Newly.renovated', 'HARDWOOD', 'High.ceilings', 'High.Ceilings', 'Hardwood.floors', 
            'Multi.Level', 'View', 'HIGH.CEILINGS', 'Green.Building', 'Hardwood', 'Renovated', 'Marble.Bath', 'Granite.Kitchen', 'Light', 
            'EXPOSED.BRICK', 'High.Ceiling', 'Skylight', 'Valet', 'renovated', 'New.construction', 'Luxury.building', 'Valet.Parking', 
           'Residents.Garden', 'Marble.Bathroom', 'Multi.level', 'Exposed.Brick', 'garden']

location = ['Subway', ]

pricing = ['No.Fee', 'Reduced.Fee', 'NO.FEE']

unused = ['Pre.War', 'Common.Outdoor.Space', 'PublicOutdoor', 'Exclusive', 'Garden.Patio', 'Outdoor.Space', 'Common.roof.deck',
          'prewar', 'LOWRISE', 'SIMPLEX', 'Loft', 'Garden', 'Prewar', 'Short.Term.Allowed', 'Guarantors.Accepted', 'Lounge', 'Actual.Apt..Photos', 
         'Live.Work', 'Duplex', 'ACTUAL.APT..PHOTOS', 'HIGHRISE', 'ROOFDECK', 'Post.War', 'Hi.Rise', 'Outdoor.Areas', 'Common.backyard', 
         'building.common.outdoor.space', 'Shares.OK']

accessibility = ['Wheelchair.Access', 'Wheelchair.Ramp']

In [68]:
train.columns[139:]

Index(['In.Unit.Washer.Dryer', 'Bike.Room', 'Live.In.Superintendent',
       'Live.in.Super', 'doorman', 'On.site.Parking.Lot', 'Cable.Satellite.TV',
       'Microwave', 'Outdoor.Areas', 'Decorative.Fireplace', 'terrace', 'Gym',
       'Sublet', 'Live.In.Super', 'Wheelchair.Ramp', 'private.balcony',
       'Private.balcony', 'Common.backyard', 'Washer...Dryer', 'Multi.level',
       'patio', 'Exposed.Brick', 'building.common.outdoor.space', 'garden',
       'Washer.Dryer.in.Unit', 'Private.Deck', 'Shares.OK', 'gym',
       'Private.outdoor.space', 'Gym.In.Building', 'Common.garden', 'state',
       'city', 'distance', 'price_scaled', 'bathrooms_scaled',
       'bedrooms_scaled', 'distance_scaled'],
      dtype='object')

In [45]:
print(train.shape)
print(test.shape)

(9846, 170)
(6564, 170)


In [46]:
train['interest_level'].value_counts()

low       3884
medium    3716
high      2246
Name: interest_level, dtype: int64

In [47]:
train.describe()

,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
count,9846.000000,9846.000000,9846.000000,9846.000000,9.846000e+03,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.0,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.

#### Data Cleaning

In [48]:
train = train[train['latitude'] > 1].reset_index(drop = True)
train = train[train['price'] < 1000000].reset_index(drop = True)

In [79]:
laundry_vars = ['Laundry.in.Building', 'Laundry.in.Unit', 'Laundry.In.Building', 'Laundry.In.Unit', 'LAUNDRY', 'Washer.in.Unit', 
                'Dryer.in.Unit', 'Laundry.Room', 'Laundry', 'On.site.laundry', 'On.site.Laundry', 'Washer.Dryer', 'Washer.Dryer.in.building', 
                'In.Unit.Washer.Dryer', 'Washer...Dryer', 'Washer.Dryer.in.Unit']
dishwasher_vars = ['Dishwasher', 'dishwasher']
gym_vars = ['Fitness.Center', 'Gym.Fitness', 'Health.Club', 'Gym', 'gym', 'Gym.In.Building']
pool_vars = ['Swimming.Pool', 'Pool', 'pool', 'Indoor.Pool']
elevator_vars = ['Elevator', 'elevator']
storage_vars = ['Storage', 'storage', 'Basement.Storage']
internet_vars = ['High.Speed.Internet', 'WiFi', 'WiFi.Access']
bike_vars = ['Bike.room', 'Bike.Room']
parking_vars = ['Parking.Space', 'Garage', 'Parking', 'On.site.Garage', 'assigned.parking.space', 'Common.parking.Garage', 'Full.Service.Garage', 
               'On.site.Parking.Lot', 'Private.parking']
deck_vars = ['Roof.Deck', 'Balcony', 'Terrace', 'Patio', 'Roof.deck', 'balcony', 'terrace', 'patio', 'private.balcony', 'Private.balcony', 
             'Private.Deck']

lounge_vars = ['Residents.Lounge', 'Lounge.room']
playroom_vars = ['Childrens.Playroom', 'Children.s.Playroom']
ac_vars = ['Central.A.C', 'Air.conditioning']
kitchen_vars = ['EAT.IN.KITCHEN','Eat.In.Kitchen']
pricing_vars = ['No.Fee', 'Reduced.Fee', 'NO.FEE']
outdoor_vars = ['Courtyard', 'Outdoor.Entertainment.Space', 'Common.garden', 'Private.Outdoor.Space', 'private.outdoor.space', 'Private.outdoor.space', 
               'garden']
pet_friendly_vars = ['Dogs.Allowed', 'Cats.Allowed', 'Pet.Friendly', 'Pets.on.approval']

service_vars = ['Doorman', 'Concierge', 'Concierge.Service', 'Full.time.doorman', 'X24.7.Concierge', 'Virtual.Doorman', 'FT.Doorman', 'doorman']
super_vars = ['LIVE.IN.SUPER', 'Live.in.superintendent', 'Live.In.Superintendent','Live.in.Super', 'Live.In.Super']
hardwood_vars = ['Hardwood.Floors', 'HARDWOOD', 'Hardwood.floors', 'Hardwood']
ceiling_vars = ['High.ceilings', 'High.Ceilings', 'HIGH.CEILINGS', 'High.Ceiling']
brick_vars = ['EXPOSED.BRICK', 'Exposed.Brick']


train['Amentity_Laundry'] = np.where(np.sum(train[laundry_vars], axis = 1) > 0, 1, 0)
train['Amentity_Dishwasher'] = np.where(np.sum(train[dishwasher_vars], axis = 1) > 0, 1, 0)

In [ ]:



niceness = ['Hardwood.Floors', 'New.Construction', 'Newly.renovated', 'HARDWOOD', 'High.ceilings', 'High.Ceilings', 'Hardwood.floors', 
            'Multi.Level', 'View', 'HIGH.CEILINGS', 'Green.Building', 'Hardwood', 'Renovated', 'Marble.Bath', 'Granite.Kitchen', 'Light', 
            'EXPOSED.BRICK', 'High.Ceiling', 'Skylight', 'Valet', 'renovated', 'New.construction', 'Luxury.building', 'Valet.Parking', 
           'Residents.Garden', 'Marble.Bathroom', 'Multi.level', 'Exposed.Brick', 'garden']



unused = ['Pre.War', 'Common.Outdoor.Space', 'PublicOutdoor', 'Exclusive', 'Garden.Patio', 'Outdoor.Space', 'Common.roof.deck',
          'prewar', 'LOWRISE', 'SIMPLEX', 'Loft', 'Garden', 'Prewar', 'Short.Term.Allowed', 'Guarantors.Accepted', 'Lounge', 'Actual.Apt..Photos', 
         'Live.Work', 'Duplex', 'ACTUAL.APT..PHOTOS', 'HIGHRISE', 'ROOFDECK', 'Post.War', 'Hi.Rise', 'Outdoor.Areas', 'Common.backyard', 
         'building.common.outdoor.space', 'Shares.OK']

accessibility = ['Wheelchair.Access', 'Wheelchair.Ramp']

In [80]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden,state,city,distance,price_scaled,bathrooms_scaled,bedrooms_scaled,distance_scaled,Amentity_Laundry
0,medium,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,New York,Manhattan,1.116940,0.029573,0.166667,0.666667,0.003266,1
1,medium,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,New York,Long Island City,1.893017,0.028895,0.166667,0.500000,0.006131,1
2,medium,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,New York,New York City,3.701883,0.022572,0.166667,0.333333,0.012807,0
3,medium,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Variable Engineering

In [69]:
amenities = ['Dining.Room', 'Pre.War', 'Laundry.in.Building', 'Dishwasher', 'Hardwood.Floors']
X = train[test_cols]
Y = train['interest_level']

knn_md = KNeighborsClassifier(n_neighbors = 5).fit(X, Y)

In [63]:
test_pred = knn_md.predict(X)

In [65]:
test_pred[0:5]

array(['medium', 'low', 'medium', 'medium', 'high'], dtype=object)

In [ ]:
## K-NN by category:
## Ex. Amenities, location benefits, pet info, safety, privacy, "niceness" of the unit (up-to-date appliances, newly renovated, etc.)

## Creating a scoring metric (0-100) for each of these categories

In [ ]:
## Extracting state and city name
def city_state(data):
    data['state'] = ''
    data['city'] = ''
    for i in range(data.shape[0]):
        location = revgc.search([data.latitude[i], data.longitude[i]])
        data.at[i, 'state'] = location[0]['admin1']
        data.at[i, 'city'] = location[0]['name']
    return data
        
## Computing distance from unit to Central Park
def central_park(data):
    data['distance'] = np.nan
    for i in range(data.shape[0]):
        lat = data.at[i, 'latitude']; long = data.at[i, 'longitude']
        data.at[i, 'distance'] = haversine((lat, long), (40.781179, -73.966098), unit = 'mi')
    return data

## Applying functions
train = city_state(train)
train = central_park(train)

test = city_state(test)
test = central_park(test)

In [ ]:
## Price per bedroom
rooms_train = train['bedrooms'].apply(lambda x: max(x, 0.5))
train['price_per_bedroom'] = train['price'] / rooms_train

rooms_test = test['bedrooms'].apply(lambda x: max(x, 0.5))
test['price_per_bedroom'] = test['price'] / rooms_test

In [58]:
## Standardizing price variable
def min_max_scaler(data, minimun, maximum):
    data_scaled = (data - minimun) / (maximum - minimun)
    return data_scaled

train['price_scaled'] = min_max_scaler(train['price'], np.min(train['price']), np.max(train['price']))
train['bathrooms_scaled'] = min_max_scaler(train['bathrooms'], np.min(train['bathrooms']), np.max(train['bathrooms']))
train['bedrooms_scaled'] = min_max_scaler(train['bedrooms'], np.min(train['bedrooms']), np.max(train['bedrooms']))
train['distance_scaled'] = min_max_scaler(train['distance'], np.min(train['distance']), np.max(train['distance']))

# test['price_scaled'] = min_max_scaler(test['price'], np.min(train['price']), np.max(train['price']))
# test['bathrooms_scaled'] = min_max_scaler(test['bathrooms'], np.min(train['bathrooms']), np.max(train['bathrooms']))
# test['bedrooms_scaled'] = min_max_scaler(test['bedrooms'], np.min(train['bedrooms']), np.max(train['bedrooms']))
# test['distance_scaled'] = min_max_scaler(test['distance'], np.min(train['distance']), np.max(train['distance']))

In [59]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden,state,city,distance,price_scaled,bathrooms_scaled,bedrooms_scaled,distance_scaled
0,medium,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,New York,Manhattan,1.116940,0.029573,0.166667,0.666667,0.003266
1,medium,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,New York,Long Island City,1.893017,0.028895,0.166667,0.500000,0.006131
2,medium,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,New York,New York City,3.701883,0.022572,0.166667,0.333333,0.012807
3,medium,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
## Rounding lat. and lomg. coordinates
train['lat_rounded'] = np.round(train['latitude'], 2)
train['long_rounded'] = np.round(train['longitude'], 2)

test['lat_rounded'] = np.round(test['latitude'], 2)
test['long_rounded'] = np.round(test['longitude'], 2)

In [ ]:
## Distance to Central Park:

40.781179, -73.966098

#### Splitting the training data into train and validation sets

#### Modelling

In [51]:
## Defining the input and target variables
X_train = train.drop(columns = ['interest_level', 'street_address'])
Y_train = train['interest_level']

X_test = test.drop(columns = ['street_address', 'ID'])

In [40]:
## Building the multi-classifier (using RF)
one_vs_all_RF = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
train_preds = one_vs_all_RF.predict_proba(X_train)
test_preds = one_vs_all_RF.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
print(train_error)

## Configuring submission
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/Submission1.csv', index = False)

In [52]:
## Building the multi-classifier (using SVM)
one_vs_all_svm = OneVsRestClassifier(estimator = LGBMClassifier()).fit(X_train, Y_train)

## Predicting on the test
train_preds = one_vs_all_svm.predict_proba(X_train)
test_preds = one_vs_all_svm.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
print(train_error)

## Configuring submission
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/Submission2.csv', index = False)

0.6387222937678705
